In [27]:
#LOADS LIBRARIES
import os
import sys
import warnings
warnings.simplefilter('ignore')
import pickle
#import dill as pickle
import json

import numpy as np
import pandas as pd
import math

import time 
import datetime as dt
from datetime import timedelta
from dateutil.relativedelta import relativedelta

In [2]:
import matplotlib.pyplot as plt
from matplotlib.ticker import PercentFormatter
import seaborn as sns

In [3]:
# Defines the storage directories
DIR = os.getcwd()
OUTPUT_DIR = os.getcwd() + '/results/'

In [5]:
params_df = pd.read_excel(DIR + '/prueba_dataanalyst.xlsx',sheet_name='parametros')
print(params_df.shape)
params_df

(25, 7)


,parametro,punto,categoria,pp_unidades_stock,unidades_stock,pp_unidades_vendidas,unidades_vendidas
0,Distribución,P1,Igual o mayor a 90 días,0.5,2500,0.3,900.0
1,Distribución,P1,Entre 75-90 días,0.2,1000,0.5,1500.0
2,Distribución,P1,Menor a 75 días,0.3,1500,0.2,600.0
3,Distribución,P2,Igual o mayor a 90 días,0.1,1200,0.6,4800.0
4,Distribución,P2,Entre 75-90 días,0.7,8400,0.2,1600.0
5,Distribución,P2,Menor a 75 días,0.2,2400,0.2,1600.0
6,Distribución,P3,Igual o mayor a 90 días,0.1,700,0.5,1000.0
7,Distribución,P3,Entre 75-90 días,0.1,700,0.4,800.0
8,Distribución,P3,Menor a 75 días,0.8,5600,0.1,200.0
9,Distribución,P4,Igual o mayor a 90 días,0.7,4200,0.1,200.0


In [6]:
distribucion_df = params_df[params_df['parametro']=='Distribución'].copy()
distribucion_df

,parametro,punto,categoria,pp_unidades_stock,unidades_stock,pp_unidades_vendidas,unidades_vendidas
0,Distribución,P1,Igual o mayor a 90 días,0.5,2500,0.3,900.0
1,Distribución,P1,Entre 75-90 días,0.2,1000,0.5,1500.0
2,Distribución,P1,Menor a 75 días,0.3,1500,0.2,600.0
3,Distribución,P2,Igual o mayor a 90 días,0.1,1200,0.6,4800.0
4,Distribución,P2,Entre 75-90 días,0.7,8400,0.2,1600.0
5,Distribución,P2,Menor a 75 días,0.2,2400,0.2,1600.0
6,Distribución,P3,Igual o mayor a 90 días,0.1,700,0.5,1000.0
7,Distribución,P3,Entre 75-90 días,0.1,700,0.4,800.0
8,Distribución,P3,Menor a 75 días,0.8,5600,0.1,200.0
9,Distribución,P4,Igual o mayor a 90 días,0.7,4200,0.1,200.0


In [74]:
current_date = dt.datetime.now().date()
p120 = current_date - timedelta(days = 120)
p90 = current_date - timedelta(days = 90)
p75 = current_date - timedelta(days = 75)

print(p120,p90,p75)

ls_p90 = [
    p120 + timedelta(days = 3),
    p120 + timedelta(days = 8),
    p120 + timedelta(days = 15),
]

ls_p75 = [
    p90 + timedelta(days = 3),
    p90 + timedelta(days = 8),
    p90 + timedelta(days = 15),
]

ls_p30 = [
    p75 + timedelta(days = 3),
    p75 + timedelta(days = 8),
    p75 + timedelta(days = 15),
]

ls_fecharepventas = [
    p120 + timedelta(days = 20),
    p90 + timedelta(days = 20),
    p75 + timedelta(days = 20),
]





2022-11-17 2022-12-17 2023-01-01


In [78]:
import random
random.seed(38)


input_df = distribucion_df.copy()

ttrows = len(input_df)
for i in range(0,ttrows):
    row_df = input_df.iloc[i,:]
    
    rows_qty = int(row_df['unidades_stock'] + row_df['unidades_vendidas'])
    
    layeridx_df = pd.DataFrame(range(0,rows_qty),columns=['layer_idx'])
    layeridx_df['categoria'] = row_df['categoria']
    layeridx_df['venta_id']=0
    layeridx_df.loc[layeridx_df['layer_idx'] >= row_df['unidades_stock'] , 'venta_id'] = 1
    
    
    layeridx_df['fecha_recepción'] = None
    layeridx_df.loc[layeridx_df['categoria'].isin(['Igual o mayor a 90 días']), 'fecha_recepción'] = ls_p90[random.randint(0, 2)]
    layeridx_df.loc[layeridx_df['categoria'].isin(['Entre 75-90 días']), 'fecha_recepción'] = ls_p75[random.randint(0, 2)]
    layeridx_df.loc[layeridx_df['categoria'].isin(['Menor a 75 días']), 'fecha_recepción'] = ls_p30[random.randint(0, 2)]
    
    layeridx_df['fecha_actual'] = current_date
    
    layeridx_df['dias_stock'] = layeridx_df['fecha_actual'] - layeridx_df['fecha_recepción']
    layeridx_df['dias_stock'] = layeridx_df['dias_stock'].apply(lambda x: x.days)
    
    
    layeridx_df['fecha_venta'] = None
    layeridx_df.loc[(layeridx_df['categoria'].isin(['Igual o mayor a 90 días']))
                    &(layeridx_df['venta_id'].isin([1]))
                    ,'fecha_venta'] = ls_fecharepventas[0]
    
    layeridx_df.loc[(layeridx_df['categoria'].isin(['Entre 75-90 días']))
                    &(layeridx_df['venta_id'].isin([1]))
                    ,'fecha_venta'] = ls_fecharepventas[1]
    
    layeridx_df.loc[(layeridx_df['categoria'].isin(['Menor a 75 días']))
                    &(layeridx_df['venta_id'].isin([1]))
                    ,'fecha_venta'] = ls_fecharepventas[2]
    
    row_df_01 = pd.merge(pd.DataFrame([row_df]),
                        layeridx_df,
                        on=['categoria'],
                        how='left')
    
    
    
    if i==0:
        row_df_02 = row_df_01.copy()
    else:
        row_df_02 = pd.concat([row_df_02,row_df_01],axis=0).copy()
    
laminas_df = row_df_02.copy()
laminas_df

,parametro,punto,categoria,pp_unidades_stock,unidades_stock,pp_unidades_vendidas,unidades_vendidas,layer_idx,venta_id,fecha_recepción,fecha_actual,dias_stock,fecha_venta
0,Distribución,P1,Igual o mayor a 90 días,0.5,2500,0.3,900.0,0,0,2022-12-02,2023-03-17,105,None
1,Distribución,P1,Igual o mayor a 90 días,0.5,2500,0.3,900.0,1,0,2022-12-02,2023-03-17,105,None
2,Distribución,P1,Igual o mayor a 90 días,0.5,2500,0.3,900.0,2,0,2022-12-02,2023-03-17,105,None
3,Distribución,P1,Igual o mayor a 90 días,0.5,2500,0.3,900.0,3,0,2022-12-02,2023-03-17,105,None
4,Distribución,P1,Igual o mayor a 90 días,0.5,2500,0.3,900.0,4,0,2022-12-02,2023-03-17,105,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1495,Distribución,P5,Menor a 75 días,0.3,1200,0.3,300.0,1495,1,2023-01-04,2023-03-17,72,2023-01-21
1496,Distribución,P5,Menor a 75 días,0.3,1200,0.3,300.0,1496,1,2023-01-04,2023-03-17,72,2023-01-21
1497,Distribución,P5,Menor a 75 días,0.3,1200,0.3,300.0,1497,1,2023-01-04,2023-03-17,72,2023-01-21
1498,Distribución,P5,Menor a 75 días,0.3,1200,0.3,300.0,1498,1,2023-01-04,2023-03-17,72,2023-01-21


In [79]:
print(row_df_02['venta_id'].value_counts(dropna=False).reset_index().sort_values(['index']))
print(row_df_02['fecha_recepción'].value_counts(dropna=False).reset_index().sort_values(['index']))
print(row_df_02['dias_stock'].value_counts(dropna=False).reset_index().sort_values(['index']))
print(row_df_02['fecha_venta'].value_counts(dropna=False).reset_index().sort_values(['index']))


summ_pf = row_df_02.groupby(['punto','categoria'],as_index=False).agg(
    min_dias_stock = ('dias_stock','min'),
    max_dias_stock = ('dias_stock','max'),
    laminas = ('dias_stock','count'),
)

summ_pf

   index  venta_id
0      0     34000
1      1     16000
        index  fecha_recepción
3  2022-11-25             7500
1  2022-12-02             9500
0  2022-12-20            14100
6  2022-12-25             1500
5  2023-01-01             2000
2  2023-01-04             9300
4  2023-01-09             6100
   index  dias_stock
4     67        6100
2     72        9300
5     75        2000
6     82        1500
0     87       14100
1    105        9500
3    112        7500
        index  fecha_venta
1  2022-12-07         7200
2  2023-01-06         4700
3  2023-01-21         4100
0        None        34000


,punto,categoria,min_dias_stock,max_dias_stock,laminas
0,P1,Entre 75-90 días,87,87,2500
1,P1,Igual o mayor a 90 días,105,105,3400
2,P1,Menor a 75 días,67,67,2100
3,P2,Entre 75-90 días,87,87,10000
4,P2,Igual o mayor a 90 días,112,112,6000
5,P2,Menor a 75 días,67,67,4000
6,P3,Entre 75-90 días,82,82,1500
7,P3,Igual o mayor a 90 días,105,105,1700
8,P3,Menor a 75 días,72,72,5800
9,P4,Entre 75-90 días,87,87,1600


In [80]:
laminas_df = laminas_df.sort_values(['punto','fecha_recepción'],ascending=[True,True])
laminas_df['layer_idx2'] = laminas_df.groupby(['punto']).cumcount() + 1

laminas_df['layer_code'] = laminas_df['punto'] + '-' + laminas_df['layer_idx2'].apply(lambda x: '{:06}'.format(x))

laminas_df

,parametro,punto,categoria,pp_unidades_stock,unidades_stock,pp_unidades_vendidas,unidades_vendidas,layer_idx,venta_id,fecha_recepción,fecha_actual,dias_stock,fecha_venta,layer_idx2,layer_code
0,Distribución,P1,Igual o mayor a 90 días,0.5,2500,0.3,900.0,0,0,2022-12-02,2023-03-17,105,None,1,P1-000001
1,Distribución,P1,Igual o mayor a 90 días,0.5,2500,0.3,900.0,1,0,2022-12-02,2023-03-17,105,None,2,P1-000002
2,Distribución,P1,Igual o mayor a 90 días,0.5,2500,0.3,900.0,2,0,2022-12-02,2023-03-17,105,None,3,P1-000003
3,Distribución,P1,Igual o mayor a 90 días,0.5,2500,0.3,900.0,3,0,2022-12-02,2023-03-17,105,None,4,P1-000004
4,Distribución,P1,Igual o mayor a 90 días,0.5,2500,0.3,900.0,4,0,2022-12-02,2023-03-17,105,None,5,P1-000005
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1495,Distribución,P5,Menor a 75 días,0.3,1200,0.3,300.0,1495,1,2023-01-04,2023-03-17,72,2023-01-21,4996,P5-004996
1496,Distribución,P5,Menor a 75 días,0.3,1200,0.3,300.0,1496,1,2023-01-04,2023-03-17,72,2023-01-21,4997,P5-004997
1497,Distribución,P5,Menor a 75 días,0.3,1200,0.3,300.0,1497,1,2023-01-04,2023-03-17,72,2023-01-21,4998,P5-004998
1498,Distribución,P5,Menor a 75 días,0.3,1200,0.3,300.0,1498,1,2023-01-04,2023-03-17,72,2023-01-21,4999,P5-004999


In [88]:
entregas_df = laminas_df[['punto','layer_code','fecha_recepción']]
deliverydates_df = entregas_df.value_counts(['fecha_recepción'],ascending=[True]).reset_index()
deliverydates_ls = deliverydates_df['fecha_recepción'].tolist()

puntos_df = entregas_df.value_counts(['punto'],ascending=[True]).reset_index()
puntos_ls = puntos_df['punto'].tolist()



for d in deliverydates_ls:
    for p in puntos_ls:
        df_output = entregas_df[(entregas_df['punto']==p)
                               &(entregas_df['fecha_recepción']==d)].copy()
        
        
        
        if len(df_output) > 0:
            
            print(df_output.shape)
            file_name = str(p) + '_entrega_' + str(d)
            print(file_name)
        
            df_output[['punto','layer_code','fecha_recepción']].to_excel(OUTPUT_DIR + file_name + '.xlsx', index=False)
        else:
            pass


(1500, 3)
P3_entrega_2022-12-25
(2000, 3)
P5_entrega_2023-01-01
(2100, 3)
P1_entrega_2023-01-09
(4000, 3)
P2_entrega_2023-01-09
(1500, 3)
P5_entrega_2022-11-25
(6000, 3)
P2_entrega_2022-11-25
(1500, 3)
P5_entrega_2023-01-04
(2000, 3)
P4_entrega_2023-01-04
(5800, 3)
P3_entrega_2023-01-04
(3400, 3)
P1_entrega_2022-12-02
(4400, 3)
P4_entrega_2022-12-02
(1700, 3)
P3_entrega_2022-12-02
(2500, 3)
P1_entrega_2022-12-20
(1600, 3)
P4_entrega_2022-12-20
(10000, 3)
P2_entrega_2022-12-20


In [92]:


ventas_df = laminas_df[laminas_df['venta_id']==1][['punto','layer_code','fecha_venta']]
paymentdates_df = ventas_df.value_counts(['fecha_venta'],ascending=[True]).reset_index()
paymentdates_ls = paymentdates_df['fecha_venta'].tolist()

puntos_df = ventas_df.value_counts(['punto'],ascending=[True]).reset_index()
puntos_ls = puntos_df['punto'].tolist()



for d in paymentdates_ls:
    for p in puntos_ls:
        df_output = ventas_df[(ventas_df['punto']==p)
                               &(ventas_df['fecha_venta']==d)].copy()
        
        
        
        if len(df_output) > 0:
            
            print(df_output.shape)
            file_name = str(p) + '_ventas_' + str(d)
            print(file_name)
        
            df_output[['punto','layer_code','fecha_venta']].to_excel(OUTPUT_DIR + file_name + '.xlsx', index=False)
        else:
            pass

(300, 3)
P5_ventas_2023-01-21
(200, 3)
P3_ventas_2023-01-21
(1400, 3)
P4_ventas_2023-01-21
(600, 3)
P1_ventas_2023-01-21
(1600, 3)
P2_ventas_2023-01-21
(400, 3)
P5_ventas_2023-01-06
(800, 3)
P3_ventas_2023-01-06
(400, 3)
P4_ventas_2023-01-06
(1500, 3)
P1_ventas_2023-01-06
(1600, 3)
P2_ventas_2023-01-06
(300, 3)
P5_ventas_2022-12-07
(1000, 3)
P3_ventas_2022-12-07
(200, 3)
P4_ventas_2022-12-07
(900, 3)
P1_ventas_2022-12-07
(4800, 3)
P2_ventas_2022-12-07
